In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
tweet_data = pd.read_feather('data/tweets_by_state.feather')
tweet_data.head()

,created_at,text,lang,full_location,country,state
0,Fri Aug 12 10:04:02 +0000 2016,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,en,"Baton Rouge, LA",United States,LA
1,Fri Aug 12 10:04:30 +0000 2016,#CNN #newday clear #Trump deliberately throwin...,en,"Baltimore, MD",United States,MD
2,Fri Aug 12 10:04:46 +0000 2016,"@realDonaldTrump, you wouldn't recognize a lie...",en,"Palm Springs, CA",United States,CA
3,Fri Aug 12 10:04:48 +0000 2016,"""Kid, you know, suing someone? Thats the most ...",en,"Secaucus, NJ",United States,NJ
4,Fri Aug 12 10:04:48 +0000 2016,@HillaryClinton you ARE the co-founder of ISIS...,en,"Irving, TX",United States,TX


In [7]:
tweet_data.loc([0])

TypeError: unhashable type: 'list'